In [1]:
import csv, sys, string, io
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import numpy as np
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.preprocessing import text, sequence
import warnings
import pickle
from sklearn.multiclass import OneVsRestClassifier
warnings.filterwarnings('ignore')

In [2]:
def getData(file):
    tsv_file = open(file)
    df = pd.read_csv(tsv_file, delimiter="\t")
    return df

trainFilename = "sub-task-1h/sub-task-1h-train-te.tsv"
validFilename = "sub-task-1h/sub-task-1h-dev-te.tsv"
testFilename = "sub-task-1h/sub-task-1h-test-te.txt"
trainDF = getData(trainFilename)
validDF = getData(validFilename)
testDF = getData(testFilename)
print("Train Data Shape: ",trainDF.shape)
print("Validation Data Shape: ",validDF.shape)
print("Test Data Shape: ",testDF.shape)

#totalTextData = pd.concat([trainDF['text'], validDF['text'], testDF['text']])
totalTextData = pd.concat([trainDF['text'], validDF['text']])
#totalTextData = pd.concat([trainDF, validDF])

Train Data Shape:  (68865, 2)
Validation Data Shape:  (5920, 2)
Test Data Shape:  (2611, 1)


In [112]:
validDF.label.value_counts() 

cse         2175
phy         1650
com_tech     970
bioche       580
other        390
mgmt         155
Name: label, dtype: int64

In [11]:
svmResultFile = "run_train_valid_test.tsv"
svmDF = getData(svmResultFile)
print("Train Data Shape: ",svmDF.shape)

Train Data Shape:  (2611, 4)


In [25]:
newSVMDF.shape
newSVMDF.to_csv('TeluguTechDOM_sub-task-1h-test-te_run-2.tsv', sep = '\t')

In [28]:
def dataPreprocess(data):
    temp = []
    punctuations = list(string.punctuation)
    digits = list(string.digits)
    letters = list(string.ascii_letters)
    #removePunctuations = str.maketrans('', '', string.punctuation)
    #removeDigits = str.maketrans('', '', string.digits)
    #removeAlphabets = str.maketrans('', '', string.ascii_letters)
    for item in data:
        item = item.split(" ")
        temp1 = []
        for token in item:
            if(len(token)>1 and token not in digits and token not in punctuations):
                temp1.append(token)
        temp1 = ' '.join(temp1)
        #print(temp1)
        #token = token.translate(removePunctuations)
        #token = token.translate(removeDigits)
        #item = item.translate(removeAlphabets)
        #print(item.split)
        temp.append(temp1)
    return temp
#print(string.digits)
trainDF['processedText'] = dataPreprocess(trainDF['text'])
validDF['processedText'] = dataPreprocess(validDF['text'])
testDF['processedText'] = dataPreprocess(testDF['text'])

In [ ]:
processedText = pd.concat([trainDF['processedText'], validDF['processedText'], testDF['processedText']])

In [6]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(totalTextData)
xtrain_tfidf =  tfidf_vect.transform(trainDF['text'])
xvalid_tfidf =  tfidf_vect.transform(validDF['text'])

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(totalTextData)
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(trainDF['text'])
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(validDF['text'])

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=15000)
tfidf_vect_ngram_chars.fit(totalTextData)
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(trainDF['text']) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(validDF['text']) 
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(testDF['text'])

In [23]:
from imblearn.over_sampling import RandomOverSampler

ROS = RandomOverSampler(random_state=11)
ros_tfidf_text, ros_tfidf_labels = ROS.fit_sample(xtrain_tfidf_ngram_chars, trainDF['label'])
print(ros_tfidf_labels.shape, ros_tfidf_text.shape)

(149622,) (149622, 15000)


In [5]:
labelEncoder = preprocessing.LabelEncoder()
labelEncoder.fit(['phy','cse','other','com_tech','bioche','mgmt'])
trainDF['numericalLabels'] = labelEncoder.transform(trainDF['label'])
validDF['numericalLabels'] = labelEncoder.transform(validDF['label'])

encoder = preprocessing.LabelEncoder()
trainLabels = encoder.fit_transform(trainDF['label'])
trainLabels = [to_categorical(i, num_classes=6) for i in trainLabels]
trainLabels = np.asarray(trainLabels)
print(trainLabels.shape)
validLabels = encoder.fit_transform(validDF['label'])
validLabels = [to_categorical(i, num_classes=6) for i in validLabels]
validLabels = np.asarray(validLabels)
print(validLabels.shape)

(68865, 6)
(5920, 6)


In [5]:
lengths = []
for item in totalTextData:
    lengths.append(len(item))
    #print(len(item))
print("Average length of texts: ", sum(lengths)/len(lengths))

Average length of texts:  112.42214347797018


In [15]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('TeluguFasttextVectors/cc.te.300.vec')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [5]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(totalTextData)
word_index = token.word_index

input_size=150
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(trainDF['text']), maxlen=input_size)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(validDF['text']), maxlen=input_size)


'''#create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

#Save to pickle        
with open('wordEmbeddingsMatrix.pickle', 'wb') as handle:
    pickle.dump(embedding_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)'''
    
#Load pickle
with open('wordEmbeddingsMatrix.pickle', 'rb') as handle:
    embedding_matrix = pickle.load(handle)

In [4]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, feature_vector_test, epoch=False, is_neural_net=False):
    if is_neural_net:
        f = open("score.txt", "w")
        classifier.fit(feature_vector_train, label, epochs=epoch)
    
        # predict the labels on validation dataset
        predictions = classifier.predict(feature_vector_valid)
        predictions1 = predictions.argmax(axis=-1)
        validLabels1 = validLabels.argmax(axis=-1)
        #print(predictions, predictions1)
        #print(validLabels, validLabels1)
        acc = metrics.accuracy_score(predictions1, validLabels1)
        f1Score = metrics.f1_score(predictions1, validLabels1, average='macro')
        classificationReport = classification_report(predictions1, validLabels1)
        f.write(str(acc)+"\n")
        f.write(str(f1Score)+"\n")
        #f.write(classifier.history)
        
    else:
        # fit the training dataset on the classifier
        classifier.fit(feature_vector_train, label)
    
        # predict the labels on validation dataset
        predictions = classifier.predict(feature_vector_valid)
        validDF['predicted_label']=predictions
        test_predictions = classifier.predict(feature_vector_test)
        testDF['predicted_label'] = test_predictions
        acc = metrics.accuracy_score(predictions, validDF['label'])
        f1Score = metrics.f1_score(predictions, validDF['label'], average='macro')
        classificationReport = classification_report(predictions, validDF['label'])
    
    return acc, f1Score, classificationReport

In [ ]:
# Naive Bayes on Character Level TF IDF Vectors
accuracy, f1_Score = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, trainDF['label'], xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy, f1_Score)

In [8]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy, f1_Score = train_model(linear_model.LogisticRegression(), xtrain_tfidf, trainDF['label'], xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy, f1_Score)

#Linear Classifier on Ngram Level TF IDF Vectors
accuracy, f1_Score = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, trainDF['label'], xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy, f1_Score)

# Linear Classifier on Character Level TF IDF Vectors
accuracy, f1_Score, classReport = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, trainDF['label'], xvalid_tfidf_ngram_chars, xtest_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy, f1_Score,"\n", classReport)

LR, CharLevel Vectors:  0.8827702702702702 0.8702677684005882 
               precision    recall  f1-score   support

      bioche       0.89      0.96      0.93       538
    com_tech       0.91      0.91      0.91       969
         cse       0.92      0.83      0.87      2391
        mgmt       0.74      0.96      0.83       119
       other       0.74      0.84      0.79       340
         phy       0.86      0.91      0.89      1563

    accuracy                           0.88      5920
   macro avg       0.84      0.90      0.87      5920
weighted avg       0.89      0.88      0.88      5920



In [9]:
validDF

,text,label,predicted_label
0,ఒకవేళ సమూహ సభ్యులు చర్చను పక్కదారి మళ్లిస్తున్...,com_tech,com_tech
1,ఈ ఆంటీబాడీలు మరియు అసైటిస్ ద్రవంలో ఉత్పత్తి చే...,bioche,bioche
2,"కాబట్టి , ఈ కవచం దరఖాస్తు చేసుకుంటే , ద్రవ కణం...",phy,phy
3,ఇది ఒక నమూనా మాత్రమే .,com_tech,cse
4,మనము రిసోర్సస్ నుండి తీసిన మాదిరిగానే ఒక సరళమ...,cse,cse
...,...,...,...
5915,VNet ఇది ఒక వర్చువల్ ప్రైవేట్ నెట్వర్క్ మరియు...,cse,cse
5916,డెల్టా ఆంగిల్ మారదు కాని ఇది షుగర్ ఆకృతీకరణపై ...,bioche,bioche
5917,ఇది విలక్షణమైన పట్టణము వంటి ఒక శిథిలమైన ఎక్స్ప...,cse,cse
5918,"ఎందుకు , మీరు నష్టాలను అధిగమించడానికి ఎందుకంటే .",phy,cse


In [30]:
model = OneVsRestClassifier(linear_model.LogisticRegression())
accuracy, f1_Score, classReport = train_model(model, xtrain_tfidf_ngram_chars, trainDF['label'], xvalid_tfidf_ngram_chars, xtest_tfidf_ngram_chars)
print("OvsRestLR, CharLevel Vectors: ", accuracy, f1_Score,"\n", classReport)

OvsRestLR, CharLevel Vectors:  0.8760135135135135 0.8647051021742347 
               precision    recall  f1-score   support

      bioche       0.90      0.97      0.93       540
    com_tech       0.91      0.90      0.91       976
         cse       0.91      0.83      0.87      2404
        mgmt       0.72      0.97      0.83       116
       other       0.71      0.87      0.78       315
         phy       0.85      0.90      0.87      1569

    accuracy                           0.88      5920
   macro avg       0.83      0.91      0.86      5920
weighted avg       0.88      0.88      0.88      5920



In [ ]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, trainDF['label'], xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

In [48]:
# RF on Word Level TF IDF Vectors
accuracy, f1_Score = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, trainDF['label'], xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy, f1_Score)

RF, WordLevel TF-IDF:  0.668918918918919 0.5761793854482674


In [ ]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
#accuracy, f1_Score = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), trainDF['label'], xvalid_tfidf.tocsc())
#print("Xgb, WordLevel TF-IDF: ", accuracy, f1_Score)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy, f1_Score = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), trainDF['label'], xvalid_tfidf_ngram_chars.tocsc(), xtest_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy, f1_Score)

In [6]:
for i in range(22,23):
    # Extereme Gradient Boosting on Character Level TF IDF Vectors
    accuracy, f1_Score, classReport = train_model(MLPClassifier(hidden_layer_sizes= (256,), learning_rate_init=0.0001, activation='relu', n_iter_no_change=True, verbose=True, random_state=1, max_iter=i), xtrain_tfidf_ngram_chars, trainDF['label'], xvalid_tfidf_ngram_chars, xtest_tfidf_ngram_chars)
    print("MLP, WordLevel TF-IDF: ",  accuracy, f1_Score,"\n", classReport)

Iteration 1, loss = 1.37803929
Iteration 2, loss = 0.77890729
Iteration 3, loss = 0.53531725
Iteration 4, loss = 0.42971023
Iteration 5, loss = 0.37098173
Iteration 6, loss = 0.33247569
Iteration 7, loss = 0.30437601
Iteration 8, loss = 0.28259671
Iteration 9, loss = 0.26479156
Iteration 10, loss = 0.24994096
Iteration 11, loss = 0.23697560
Iteration 12, loss = 0.22569611
Iteration 13, loss = 0.21564287
Iteration 14, loss = 0.20655091
Iteration 15, loss = 0.19841531
Iteration 16, loss = 0.19093693
Iteration 17, loss = 0.18393985
Iteration 18, loss = 0.17766783
Iteration 19, loss = 0.17169291
Iteration 20, loss = 0.16617635
Iteration 21, loss = 0.16099913
Iteration 22, loss = 0.15623947
MLP, WordLevel TF-IDF:  0.9003378378378378 0.8936647524524427 
               precision    recall  f1-score   support

      bioche       0.94      0.96      0.95       567
    com_tech       0.93      0.94      0.93       961
         cse       0.91      0.87      0.89      2259
        mgmt       0.83 

In [9]:
testDF.predicted_label.value_counts(dropna=False)

bioche      863
cse         594
phy         355
com_tech    297
other       269
mgmt        233
Name: predicted_label, dtype: int64

In [7]:
testDF.predicted_label.value_counts(dropna=False)

bioche      871
cse         579
phy         352
com_tech    300
other       269
mgmt        240
Name: predicted_label, dtype: int64

In [8]:
testDF.to_csv('TeluguTechDOM_sub-task-1h-test-te_run-3.tsv', sep = '\t')

In [24]:
testDF.predicted_label.value_counts(dropna=False)

bioche      681
cse         471
phy         286
com_tech    242
other       217
mgmt        201
Name: predicted_label, dtype: int64

In [15]:
testDF.predicted_label.value_counts(dropna=False)

bioche      668
cse         501
phy         289
com_tech    246
other       212
mgmt        182
Name: predicted_label, dtype: int64

In [10]:
def create_rnn_lstm(input_size):    
    # Add an Input Layer
    input_layer = layers.Input((input_size, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    #embedding_layer = layers.SpatialDropout1D(0.25)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(150)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(150, activation="tanh")(lstm_layer)
    #output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(6, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy')
    
    return model

classifier = create_rnn_lstm(input_size)
accuracy = train_model(classifier, train_seq_x, trainLabels, valid_seq_x,1, is_neural_net=True)
print("RNN-LSTM, Word Embeddings",  accuracy)

2153/2153 [==============================] - 510s 237ms/step - loss: 0.7274
RNN-LSTM, Word Embeddings (0.7826013513513513, 0.7705641380399767)


In [24]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints



class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [ ]:
def create_rnn_bilstm_attention(input_size):    
    # Add an Input Layer
    input_layer = layers.Input((input_size, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    #embedding_layer = layers.SpatialDropout1D(0.2)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.LSTM(300, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(embedding_layer)
    
    #Self Attention
    attention_layer = Attention(input_size)(lstm_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(256, activation="relu")(attention_layer)
    #output_layer1 = layers.Dropout(0.2)(output_layer1)
    output_layer2 = layers.Dense(6, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy')
    
    return model

classifier = create_rnn_bilstm_attention(input_size)
accuracy = train_model(classifier, train_seq_x, trainLabels, valid_seq_x,5, is_neural_net=True)
print("RNN-LSTM, Word Embeddings",  accuracy)

In [14]:
def create_cnn(input_size):
    # Add an Input Layer
    input_layer = layers.Input((input_size, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.25)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(256, 3, activation="tanh")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(128, activation="tanh")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(6, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy')
    
    return model

classifier = create_cnn(input_size)
accuracy = train_model(classifier, train_seq_x, trainLabels, valid_seq_x,25, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/25
2153/2153 [==============================] - 144s 67ms/step - loss: 0.6870
Epoch 2/25
2153/2153 [==============================] - 161s 75ms/step - loss: 0.4975
Epoch 3/25
2153/2153 [==============================] - 163s 76ms/step - loss: 0.4265
Epoch 4/25
2153/2153 [==============================] - 159s 74ms/step - loss: 0.3793
Epoch 5/25
2153/2153 [==============================] - 159s 74ms/step - loss: 0.3439
Epoch 6/25
2153/2153 [==============================] - 169s 79ms/step - loss: 0.3187
Epoch 7/25
2153/2153 [==============================] - 157s 73ms/step - loss: 0.2926
Epoch 8/25
2153/2153 [==============================] - 156s 73ms/step - loss: 0.2775
Epoch 9/25
2153/2153 [==============================] - 157s 73ms/step - loss: 0.2683
Epoch 10/25
2153/2153 [==============================] - 157s 73ms/step - loss: 0.2502
Epoch 11/25
2153/2153 [==============================] - 33530s 16s/step - loss: 0.2365
Epoch 12/25
2153/2153 [============================

In [45]:
def create_cnn(input_size):
    # Add an Input Layer
    input_layer = layers.Input((input_size, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(300, 5, activation="relu")(embedding_layer)
    # Add the pooling Layer
    pooling_layer = layers.MaxPool1D()(conv_layer)
     # Add the convolutional Layer
    conv_layer = layers.Convolution1D(256, 3, activation="relu")(pooling_layer)
    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(256, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(6, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy')
    
    return model

classifier = create_cnn(input_size)
accuracy = train_model(classifier, train_seq_x, trainLabels, valid_seq_x,50, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.7384
Epoch 2/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.5436
Epoch 3/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.4617
Epoch 4/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.3996
Epoch 5/50
2153/2153 [==============================] - 203s 94ms/step - loss: 0.3551
Epoch 6/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.3182
Epoch 7/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.2878
Epoch 8/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.2650
Epoch 9/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.2491
Epoch 10/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.2286
Epoch 11/50
2153/2153 [==============================] - 204s 95ms/step - loss: 0.2127
Epoch 12/50
2153/2153 [=============================

In [13]:
def create_cnn(input_size):
    # channel1
    input_layer1 = layers.Input((input_size, ))
    embedding_layer1 = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer1)
    conv_layer1 = layers.Convolution1D(filters=256, kernel_size=3, activation="tanh")(embedding_layer1)
    drop_layer1 = layers.Dropout(0.25)(conv_layer1)
    pooling_layer1 = layers.MaxPooling1D()(drop_layer1)
    flaten_layer1 = layers.Flatten()(pooling_layer1)
    
    # channel2
    input_layer2 = layers.Input((input_size, ))
    embedding_layer2 = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer2)
    conv_layer2 = layers.Convolution1D(filters=256, kernel_size=5, activation="tanh")(embedding_layer2)
    drop_layer2 = layers.Dropout(0.25)(conv_layer2)
    pooling_layer2 = layers.MaxPooling1D()(drop_layer2)
    flaten_layer2 = layers.Flatten()(pooling_layer2)
    
    # channel3
    input_layer3 = layers.Input((input_size, ))
    embedding_layer3 = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer3)
    conv_layer3 = layers.Convolution1D(filters=256, kernel_size=7, activation="relu")(embedding_layer3)
    drop_layer3 = layers.Dropout(0.25)(conv_layer3)
    pooling_layer3 = layers.MaxPooling1D()(drop_layer3)
    flaten_layer3 = layers.Flatten()(pooling_layer3)
    
    # merge
    merged_layer = layers.concatenate([flaten_layer1, flaten_layer2, flaten_layer3])
    # interpretation
    dense_layer = layers.Dense(128, activation='relu')(merged_layer)
    outputs = layers.Dense(6, activation='softmax')(dense_layer)
    model = models.Model(inputs=[input_layer1, input_layer2, input_layer3], outputs=outputs)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy')
    
    return model

classifier = create_cnn(input_size)
accuracy = train_model(classifier, [train_seq_x,train_seq_x,train_seq_x] , trainLabels, [valid_seq_x, valid_seq_x, valid_seq_x], 25, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/25
2153/2153 [==============================] - 178s 83ms/step - loss: 0.7408
Epoch 2/25
2153/2153 [==============================] - 168s 78ms/step - loss: 0.5681
Epoch 3/25
2153/2153 [==============================] - 166s 77ms/step - loss: 0.5080
Epoch 4/25
2153/2153 [==============================] - 165s 77ms/step - loss: 0.4608
Epoch 5/25
2153/2153 [==============================] - 166s 77ms/step - loss: 0.4267
Epoch 6/25
2153/2153 [==============================] - 166s 77ms/step - loss: 0.3993
Epoch 7/25
2153/2153 [==============================] - 166s 77ms/step - loss: 0.3801
Epoch 8/25
2153/2153 [==============================] - 166s 77ms/step - loss: 0.3594
Epoch 9/25
2153/2153 [==============================] - 166s 77ms/step - loss: 0.3429
Epoch 10/25
2153/2153 [==============================] - 167s 77ms/step - loss: 0.3301
Epoch 11/25
2153/2153 [==============================] - 167s 78ms/step - loss: 0.3155
Epoch 12/25
2153/2153 [=============================

In [ ]:
# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	plot_model(model, show_shapes=True, to_file='multichannel.png')
	return model

In [25]:
def create_rcnn(input_size=100):

    # Add an Input Layer
    input_layer = layers.Input((input_size, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.2)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.LSTM(256, return_sequences=True, dropout=0.25,recurrent_dropout=0.25))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(256, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(128, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(6, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy')
    
    return model

classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, trainLabels, valid_seq_x, 25, is_neural_net=True)
print("RCNN, Word Embeddings",  accuracy)

Epoch 1/25
2153/2153 [==============================] - 78s 36ms/step - loss: 0.7048
Epoch 2/25
2153/2153 [==============================] - 77s 36ms/step - loss: 0.4910
Epoch 3/25
2153/2153 [==============================] - 76s 35ms/step - loss: 0.4149
Epoch 4/25
2153/2153 [==============================] - 76s 35ms/step - loss: 0.3599
Epoch 5/25
2153/2153 [==============================] - 76s 35ms/step - loss: 0.3263
Epoch 6/25
2153/2153 [==============================] - 76s 35ms/step - loss: 0.2996
Epoch 7/25
2153/2153 [==============================] - 76s 35ms/step - loss: 0.2695
Epoch 8/25
2153/2153 [==============================] - 75s 35ms/step - loss: 0.2509
Epoch 9/25
2153/2153 [==============================] - 76s 36ms/step - loss: 0.2360
Epoch 10/25
2153/2153 [==============================] - 76s 35ms/step - loss: 0.2183
Epoch 11/25
2153/2153 [==============================] - 76s 35ms/step - loss: 0.2079
Epoch 12/25
2153/2153 [==============================] - 76s 35